In [106]:
import pandas as pd
import numpy as np
import os



In [107]:
data_vulnerabilities = pd.read_json(
    "dataset/vulnerabilities.json")


In [108]:
data_vulnerabilities.head()

,name,path,source,vulnerabilities
0,FibonacciBalance.sol,dataset/access_control/FibonacciBalance.sol,https://github.com/sigp/solidity-security-blog,"[{'lines': [31], 'category': 'access_control'}..."
1,arbitrary_location_write_simple.sol,dataset/access_control/arbitrary_location_writ...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [27], 'category': 'access_control'}]"
2,incorrect_constructor_name1.sol,dataset/access_control/incorrect_constructor_n...,https://github.com/trailofbits/not-so-smart-co...,"[{'lines': [20], 'category': 'access_control'}]"
3,incorrect_constructor_name2.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [18], 'category': 'access_control'}]"
4,incorrect_constructor_name3.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [17], 'category': 'access_control'}]"


In [109]:
data_vulnerabilities.vulnerabilities[0]


[{'lines': [31], 'category': 'access_control'},
 {'lines': [38], 'category': 'access_control'}]

In [110]:
all_categories = []
for idx, row in data_vulnerabilities.iterrows():
    for i in row['vulnerabilities']:
        # print(i)
        all_categories.append(i['category'])
unique_categories = set(all_categories)

In [111]:
unique_categories

{'access_control',
 'arithmetic',
 'bad_randomness',
 'denial_of_service',
 'front_running',
 'other',
 'reentrancy',
 'short_addresses',
 'time_manipulation',
 'unchecked_low_level_calls'}

In [112]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoded_category = label_encoder.fit_transform(list(unique_categories))

In [113]:
def parse_clean_Data(data, filename):
    indexes_to_drop = []
    for i, d in enumerate(data):
        # Convert bytes to string
        data[i] = data[i].decode("utf-8")
        d = d.decode("utf-8")
        # print(type(d))
        if d.startswith("/*"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith("*") or d.__contains__("@"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith(" */"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.startswith("//"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.__contains__("//"):
            data[i] = data[i].split("//")[0]
    # print(indexes_to_drop)
    # result = {f"{filename}":data}
    return data
    # data = data.dropna()
    # data = data.reset_index(drop=True)
    # return data


In [114]:
list_of_dataset=os.listdir("dataset/")
#Loop to check location is directory or not
final_output = {}
for entry in list_of_dataset:
    # Create full path
    fullpath = os.path.join("dataset/", entry)
    # If entry is a directory then get the list of files in this directory 
    if os.path.isdir(fullpath):
        print(entry)
        filenames_list = os.listdir(fullpath)
        for filename in filenames_list:
            if(filename!="README.md"):
            # print(filename)
            # Read the file content and print it
                with open(os.path.join(fullpath, filename), 'rb') as f:
                    file_Data = f.read().splitlines()
                    # final_output.append(parse_clean_Data(file_Data, filename))
                    final_output[filename] = parse_clean_Data(file_Data, filename)
                # print()
# print(final_output)
    

bad_randomness
reentrancy
unchecked_low_level_calls
other
short_addresses
denial_of_service
front_running
access_control
arithmetic
time_manipulation


In [115]:
all_filenames = []
all_codes = []
all_categories = []
for filename,v in final_output.items():
    temp_V= list(enumerate(v))
    #apply lambda function on list of tuples
    temp_V = list(map(lambda x: (x[0]+1, x[1]), temp_V))
    #get the index for the value in the dataframe
    # print(filename)
    index = data_vulnerabilities[data_vulnerabilities['name'] == filename].index[0]
    for (j,k) in temp_V:    
        for vuln in data_vulnerabilities.vulnerabilities[index]:
            # print(i)
            if j== vuln["lines"][0]:
                all_categories.append(vuln['category'])
                all_codes.append(k)
                all_filenames.append(filename)
            else:
                all_categories.append("No Vulnerability")
                # all_codes.append(temp_V)
                # all_filenames.append(filename)
                all_codes.append(k)
                all_filenames.append(filename)
    # print(index)


In [116]:
final_df = pd.DataFrame(list(zip(all_filenames, all_codes, all_categories)),columns=['filename', 'code', 'category'])

In [117]:
df1=final_df[final_df["category"]=="No Vulnerability"]
df1 = df1.drop_duplicates(subset=['code'], keep='first')
df1

,filename,code,category
0,random_number_generator.sol,,No Vulnerability
24,random_number_generator.sol,pragma solidity ^0.4.25;,No Vulnerability
36,random_number_generator.sol,contract RandomNumberGenerator {,No Vulnerability
40,random_number_generator.sol,,No Vulnerability
45,random_number_generator.sol,uint256 private salt = block.timestamp;,No Vulnerability
...,...,...,...
21318,roulette.sol,require(now != pastBlockTime);,No Vulnerability
21321,roulette.sol,pastBlockTime = now;,No Vulnerability
21323,roulette.sol,if(now % 15 == 0) {,No Vulnerability
21341,timed_crowdsale.sol,contract TimedCrowdsale {,No Vulnerability


In [118]:
df2 = final_df[final_df["category"]!="No Vulnerability"]
final_df = pd.concat([df1, df2])
print(final_df)

                                            filename  \
0                        random_number_generator.sol   
24                       random_number_generator.sol   
36                       random_number_generator.sol   
40                       random_number_generator.sol   
45                       random_number_generator.sol   
...                                              ...   
21221  integer_overflow_multitx_onefunc_feasible.sol   
21266                                ether_lotto.sol   
21317                                   roulette.sol   
21322                                   roulette.sol   
21345                            timed_crowdsale.sol   

                                                    code           category  
0                                                          No Vulnerability  
24                              pragma solidity ^0.4.25;   No Vulnerability  
36                      contract RandomNumberGenerator {   No Vulnerability  
40             

In [119]:
final_df.reset_index(drop=True, inplace=True)


In [120]:
final_df

,filename,code,category
0,random_number_generator.sol,,No Vulnerability
1,random_number_generator.sol,pragma solidity ^0.4.25;,No Vulnerability
2,random_number_generator.sol,contract RandomNumberGenerator {,No Vulnerability
3,random_number_generator.sol,,No Vulnerability
4,random_number_generator.sol,uint256 private salt = block.timestamp;,No Vulnerability
...,...,...,...
2809,integer_overflow_multitx_onefunc_feasible.sol,count -= input;,arithmetic
2810,ether_lotto.sol,var random = uint(sha3(block.timestamp...,time_manipulation
2811,roulette.sol,require(now != pastBlockTime);,time_manipulation
2812,roulette.sol,pastBlockTime = now;,time_manipulation


In [121]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
under_sampleer = RandomUnderSampler(sampling_strategy='majority')
data,y =under_sampleer.fit_resample(final_df,final_df["category"])

In [122]:
data.shape

(156, 3)

In [123]:
type(data)

pandas.core.frame.DataFrame

In [128]:
data["category"].value_counts()

unchecked_low_level_calls    53
reentrancy                   26
bad_randomness               24
arithmetic                   21
access_control               14
denial_of_service             5
front_running                 5
time_manipulation             4
other                         2
No Vulnerability              1
short_addresses               1
Name: category, dtype: int64

In [137]:
#import TfidfVectorizer from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
#initialize TfidfVectorizer 
tfidf_vectorizer=TfidfVectorizer(ngram_range=(1,2))
#convert the text into a matrix of TF-IDF features
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(data['code']).todense()

In [138]:
tfidf_vectorizer_vectors.shape

(156, 410)

In [139]:
from sklearn.preprocessing import LabelEncoder
l_e = LabelEncoder()
data['category'] = l_e.fit_transform(data['category'])

In [140]:
data

,filename,code,category
0,0x19cf8481ea15427a98ba3cdd6d9e14690011ab10.sol,require(balances[_from] >= _value); ...,0
1,multiowned_vulnerable.sol,function newOwner(address _owner) external r...,1
2,phishable.sol,require(tx.origin == owner);,1
3,incorrect_constructor_name1.sol,function IamMissing(),1
4,wallet_03_wrong_constructor.sol,function initWallet() public {,1
...,...,...,...
151,0xa1fceeff3acc57d257b917e30c4df661401d6431.sol,"contract_address.call(id, msg.send...",10
152,0x9d06cbafa865037a01d322d3f4222fa3e04e5488.sol,owner2.send(sum2);,10
153,0x9d06cbafa865037a01d322d3f4222fa3e04e5488.sol,owner.send(balance);,10
154,0xe894d54dca59cb53fe9cbc5155093605c7068220.sol,"caddress.call(id,from,_tos[i],_val...",10


In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectorizer_vectors, data['category'], test_size=0.2, random_state=42)


In [147]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100,max_iter=10000000)
lr.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


LogisticRegression(C=100, max_iter=10000000)

In [148]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test, lr.predict(X_test))
print(classification_report(y_test, lr.predict(X_test)))
# lr.predict(tfidf_vectorizer_vectors)

              precision    recall  f1-score   support

           1       0.50      1.00      0.67         2
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         3
           5       0.00      0.00      0.00         2
           7       1.00      0.75      0.86         8
           8       0.00      0.00      0.00         1
           9       1.00      1.00      1.00         1
          10       0.73      1.00      0.84         8

    accuracy                           0.84        32
   macro avg       0.65      0.72      0.67        32
weighted avg       0.81      0.84      0.81        32



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

In [144]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
# dummy_clf.predict(X_test)
print(classification_report(y_test, dummy_clf.predict(X_test)))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         8
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          10       0.25      1.00      0.40         8

    accuracy                           0.25        32
   macro avg       0.03      0.12      0.05        32
weighted avg       0.06      0.25      0.10        32



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [161]:
lr.predict(tfidf_vectorizer.transform([final_df['code'][4]]).todense())


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


array([3])

In [163]:
data["category"].value_counts()

10    53
7     26
3     24
2     21
1     14
4      5
5      5
9      4
6      2
0      1
8      1
Name: category, dtype: int64

In [162]:
final_df['code'][4]

'  uint256 private salt =  block.timestamp;'